# Earthquake risk in Greece

In [ ]:
%reload_ext autoreload
%autoreload 2

## Constraints

+ Only cells with the comment `# NOTE: Fill me!` should be filled
+ The solution only requires packages listed in the `requirements/requirements.txt`
+ When requested, tests must automatically be detected running `pytest`
+ Requested method signature should be inferred from this notebook


## Setup

In a Python >= 3.8 virtual env, run:

In [ ]:
! pip install -r ../requirements/requirements.txt
! pip install --no-deps -e ..

## Tests

In [ ]:
! cd .. && pytest

---

# Context

A client asks for an insurance of their asset, located at `(35.025, 25.763)` in Greece.

The client wishes to receive a payout under the following conditions:

+ earthquake of magnitude 4.5 or higher within 10km: full payout
+ earthquake of magnitude 5.5 or higher within 50km: 75% payout
+ earthquake of magnitude 6.5 or higher within 200km: 50% payout

In the event of aftershocks, a payout can only occur once a year using the maximal value.

## Example

If in the same year:

* an earthquake of magnitude 6.8 occurs within 200km
* **and** an aftershock of magnitude 5.8 occurs within 50km

the client receives a 75% payout, and not a 125% payout.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

from earthquakes.tools import (
    DISTANCE_COLUMN,
    LATITUDE_COLUMN,
    LONGITUDE_COLUMN,
) 

# Earthquake data

The US Geological Service (USGS) provides an [API](https://earthquake.usgs.gov/fdsnws/event/1/) to retrieve
earthquake information.

In the module `earthquakes.usgs_api`:
+ Implement the function `get_earthquake_data`,
+ The function will retrieve the earthquake data of the area of interest for the past 200 years,
+ The implementation must use the `urllib` python package,
+ The API request url must be build in a dedicated function `build_api_url`,
+ Tests should be provided for `build_api_url`.

Note: Earthquakes after the 21-10-2021 should not be considered.

In [ ]:
from earthquakes.usgs_api import get_earthquake_data

In [ ]:
# NOTE: Fill me!

In [ ]:
# NOTE: This request may take significant time (>10s)
earthquake_data = get_earthquake_data(
    latitude=latitude,
    longitude=longitude,
    radius=radius,
    minimum_magnitude=magnitude_threshold,
    end_date=datetime(year=2021, month=10, day=21)
)

In [ ]:
expected_earthquake_data = pd.DataFrame([
        ["2021-10-20T02:44:04.736Z", 35.0391, 25.2584, 7.15, 4.3, "mb", np.nan, 122.0, 0.390, 0.84, "us", "us6000fw0x", "2021-10-20T12:18:19.580Z", "10 km ENE of Pýrgos, Greece", "earthquake", 6.1, 5.7, 0.113, 22.0, "reviewed", "us", "us"],
        ["2021-10-15T23:25:43.051Z", 34.8943, 26.3729, 12.08, 4.1, "mb", np.nan, 71.0, 0.916, 0.70, "us", "us6000fv2i", "2021-10-17T12:44:11.538Z", "35 km SSE of Palekastro, Greece", "earthquake", 3.3, 5.1, 0.114, 21.0, "reviewed", "us", "us"],
        ["2021-10-12T09:36:26.949Z", 34.8373, 26.3556, 10.00, 4.4, "mb", np.nan, 175.0, 0.967, 0.80, "us", "us6000fuy1", "2021-10-19T08:52:12.040Z", "41 km SSE of Palekastro, Greece", "earthquake", 3.2, 1.9, 0.311, 3.0, "reviewed", "us", "us"],
        ["2021-10-12T09:24:03.839Z", 35.1931, 26.2556, 10.00, 6.4, "mww", np.nan, 20.0, 0.820, 0.73, "us", "us6000ftxu", "2021-10-17T17:55:29.061Z", "0 km SSE of Palekastro, Greece", "earthquake", 5.0, 1.8, 0.051, 37.0, "reviewed", "us", "us"],
        ["2021-10-11T14:43:20.315Z", 34.1319, 26.0136, 10.00, 4.4, "mb", np.nan, 95.0, 1.478, 1.03, "us", "us6000fuwi", "2021-10-17T20:21:42.040Z", "100 km SSE of Ierápetra, Greece", "earthquake", 6.2, 1.9, 0.155, 13.0, "reviewed", "us", "us"],
    ],
    columns=['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst', 'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type', 'horizontalError', 'depthError', 'magError', 'magNst', 'status', 'locationSource', 'magSource']
)

In [ ]:
assert isinstance(earthquake_data, pd.DataFrame)
assert len(earthquake_data) == 2404
pd.testing.assert_frame_equal(earthquake_data.head(5), expected_earthquake_data)

# Distance

We wish to compute the the historical payouts (i.e. the payouts that would have occurred for the past 200 years).

To compute the historical payouts, we need to know the distance between each earthquake and our client's asset.

The distance between two points on a sphere is the [Haversine distance](https://en.wikipedia.org/wiki/Haversine_formula). In the module `eathquakes.tools`:
- Implement and test the function `get_haversine_distance`,
- Use `earthquakes.tools.EARTH_RADIUS` (6378km) as an approximation of the radius of Earth.

In [ ]:
from earthquakes.tools import get_haversine_distance

distances = get_haversine_distance(earthquake_data[LATITUDE_COLUMN], earthquake_data[LONGITUDE_COLUMN], latitude, longitude)

earthquake_data[DISTANCE_COLUMN] = distances

# Historical payouts and burning costs

The historical payouts are a map `year -> payout in %`.

This map can take the form of a python `dict`, or of a pandas `Series` or `DataFrame`. 

The burning cost is the sum of payouts over a time range, expressed in %. 

The payout structure is:
| Radius | Magnitude | Payout |
|--------|-----------|--------|
| 10km   | 4.5       | 100 %  |
| 50km   | 5.5       |  75 %  |
| 200km  | 6.5       |  50 %  |

Payouts can occur only once in a given year.

In the module `earthquakes.tools`:
+ Implement the functions `compute_payouts` and `compute_burning_cost`,
+ Tests for these functions are not required.

## Example

A payout `{1950: 50, 1992: 75}` means that we would have paid our client
+ in 1950, for 50% of the insured amount (called 'limit')
+ in 1992, for 75% of the limit

Our burning cost over the 1922-2021 period would be 1.25%

In [ ]:
from earthquakes.tools import compute_payouts, compute_burning_cost

In [ ]:
# NOTE: Fill me!

In [ ]:
payouts = compute_payouts(earthquake_data, payout_structure)

In [ ]:
burning_cost = compute_burning_cost(payouts, start_year=1922, end_year=2021)

In [ ]:
np.testing.assert_allclose(burning_cost, 9.25, atol=1e-2)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
years = range(1921, 2021)
plt.plot(
    years, 
    [
        compute_burning_cost(payouts, start_year=start_year, end_year=2021) 
        for start_year in years
    ]
)
plt.gca().invert_xaxis()

# Large asset portfolio - async requests

Our client also whishes to cover a large amount of properties all over Europe.

In order to speed-up the requests to the USGS API, in the module `earthquakes.usgs_api`:
- Implement the `async` function `get_earthquake_data_for_multiple_locations`,
- The implementation should use the `asyncio` and `aiohttp` libraries,
- The solution should re-use some of the functions already written,
- Tests are not required for any of the functions.

Note: it is possible that the notebook autoreload feature doesn't work for `async` functions - a kernel restart may be necessary after each modifications.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

from earthquakes.tools import (
    LATITUDE_COLUMN,
    LONGITUDE_COLUMN,
)

In [ ]:
number_of_assets = 10
# NOTE: limiting to number of assets so that the query doesn't take too long.

In [ ]:
random_state = np.random.RandomState(0)

random_values = random_state.random(2*number_of_assets)

latitudes = random_values[::2] * 20 + 35.0
longitudes = random_values[1::2] * 25 + 3.0

In [ ]:
from earthquakes.usgs_api import get_earthquake_data_for_multiple_locations

In [ ]:
# NOTE: Fill me!

In [ ]:
# NOTE: This request may take significant time (>10s)
earthquake_data = await get_earthquake_data_for_multiple_locations(  # type: ignore
    assets, 
    radius=200, 
    minimum_magnitude=4.0, 
    end_date=datetime(year=2021, month=10, day=21)
)

In [ ]:
index = (45.97627007854649, 20.879734159310487)
assert isinstance(earthquake_data[index], pd.DataFrame)